# Определение рыночной цены автомобиля.

### Введение.


Сервис по продаже автомобилей с пробегом разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Задача - построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

**Описание данных:**

Признаки:    
•	DateCrawled — дата скачивания анкеты из базы  
•	VehicleType — тип автомобильного кузова   
•	RegistrationYear — год регистрации автомобиля  
•	Gearbox — тип коробки передач  
•	Power — мощность (л. с.)  
•	Model — модель автомобиля  
•	Kilometer — пробег (км)  
•	RegistrationMonth — месяц регистрации автомобиля  
•	FuelType — тип топлива  
•	Brand — марка автомобиля  
•	NotRepaired — была машина в ремонте или нет  
•	DateCreated — дата создания анкеты  
•	NumberOfPictures — количество фотографий автомобиля  
•	PostalCode — почтовый индекс владельца анкеты (пользователя)  
•	LastSeen — дата последней активности пользователя  
Целевой признак:  
Price — цена (евро) 

План работы:  
1.	Загрузка и подготовка данных.  
2.	Создание и обучение разных моделей с различными гиперпараметрами.  
3.	Анализ скорости работы и качества моделей (метрика - RMSE). 

### 1. Подготовка данных

In [1]:
# импорт библиотек и опций
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
# загрузка данных
data = pd.read_csv('/datasets/autos.csv')

In [3]:
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


In [5]:
data.duplicated().sum()

4

In [6]:
# удалим дупликаты
data = data.drop_duplicates().reset_index(drop=True)

Теперь займемся пропусками и аномальными значениями данных.  
Прежде всего удалим те наблюдения, где пропущены данные, критичные для определения цены, - это модель и год производства автомобиля (здесь - год регистрации).

In [7]:
data['Model'].value_counts()

golf                  29232
other                 24420
3er                   19761
polo                  13066
corsa                 12570
                      ...  
serie_2                   8
rangerover                4
serie_3                   4
range_rover_evoque        2
serie_1                   2
Name: Model, Length: 250, dtype: int64

In [8]:
# удалим наблюдения, для которых не указана модель
len1 = len(data)
data = data.dropna(subset=['Model']).reset_index(drop=True)
print ('Удалено, %:', round((len1-len(data))/len1*100,1))

Удалено, %: 5.6


Удалим наблюдения, где дата регистрации автомобиля превышает дату создания объявления. В этом случае можно утверждать, что нам возраст автомобиля неизвестен.

In [9]:
# объединим RegistrationYear и RegistrationMonth - создадим признак RegYM
data['RegYM'] = data['RegistrationYear'] + data['RegistrationMonth']/12

In [10]:
# переведем DateCreated из типа 'object' в тип 'datetime'
data['DateCreated'] = pd.to_datetime(data['DateCreated'], format='%Y-%m-%d')
# и представим в виде десятичного значения года (в таком виде уже представлен признак RegYM - год и месяц регистрации автомобиля)
data['DateCreated'] = pd.DatetimeIndex(data['DateCreated']).year+pd.DatetimeIndex(data['DateCreated']).month/12

In [11]:
# удалим наблюдения, где дата регистрации автомобиля превышает дату создания объявления
len2 = len(data)
data = data[data['RegYM']<data['DateCreated']]
print ('Удалено, %:', round((len2-len(data))/len1*100,1))

Удалено, %: 5.0


Удалим наблюдения, в которых автомобиль был зарегистрирован ранее даты нижней границы диаграммы размаха. Т.е. удалим выбросы по дате регистрации.

In [12]:
# вычислим пределы нижнего нормального значения даты регистрации
k = 1.5 # коэффициент IQR
q1 = data['RegYM'].quantile(0.25) # квартиль 25%
q3 = data['RegYM'].quantile(0.75) # квартиль 75%
iqr = q3 - q1 # межквартильный интервал
low = q1 - k*iqr # минимальное нормальное значение
low

1987.125

In [13]:
# удалим выбросы по дате регистрации
len3 = len(data)
data = data[data['RegYM']>low]
print ('Удалено, %:', round((len3-len(data))/len1*100,1))

Удалено, %: 1.5


Теперь займемся пропусками и аномальными значениями в других признаках. Но прежде, несколько преобразуем таблицу.

In [14]:
# объединим Model и Brand - создадим признак ModelBrand
data['ModelBrand'] = data['Brand']+str('_')+data['Model']

In [15]:
# удалим признаки, которые не оказывают влияние на формирование цены,
# а также столбцы RegistrationYear, RegistrationMonth, Model и Brand
data = data.drop(columns=['DateCrawled','DateCreated','NumberOfPictures','PostalCode','LastSeen', 'RegistrationYear', 'RegistrationMonth','Model','Brand'])
data.head()

,Price,VehicleType,Gearbox,Power,Kilometer,FuelType,NotRepaired,RegYM,ModelBrand
0,480,NaN,manual,0,150000,petrol,NaN,1993.000000,volkswagen_golf
1,9800,suv,auto,163,125000,gasoline,NaN,2004.666667,jeep_grand
2,1500,small,manual,75,150000,petrol,no,2001.500000,volkswagen_golf
3,3600,small,manual,69,90000,gasoline,no,2008.583333,skoda_fabia
4,650,sedan,manual,102,150000,petrol,yes,1995.833333,bmw_3er


In [16]:
# проверим разброс значений цены
print(data['Price'].min(),'...',data['Price'].max())

0 ... 20000


-требуется обработка аномальных значений цены.

In [17]:
# проверим значения типа кузова
data['VehicleType'].value_counts()

sedan          86230
small          75893
wagon          62763
bus            27507
convertible    18742
coupe          14371
suv            10964
other           2606
Name: VehicleType, dtype: int64

-смысловых дупликатов нет; присутствует значение 'other', но не будем его обрабатывать, чтобы не искажать данные.

In [18]:
# проверим разброс значений мощности
print(data['Power'].min(),'...',data['Power'].max())

0 ... 20000


-требуется обработка аномальных значений мощности.

In [19]:
# проверим разброс значений пробега
print(data['Kilometer'].min(),'...',data['Kilometer'].max())

5000 ... 150000


In [20]:
# проверим значения типа топлива
data['FuelType'].value_counts()

petrol      195136
gasoline     92456
lpg           4734
cng            511
hybrid         195
other          100
electric        63
Name: FuelType, dtype: int64

-смысловых дупликатов нет; присутствует значение 'other', но не будем его обрабатывать, чтобы не искажать данные.

Для каждой модели удалим наблюдения с аномальными значениями цены (лежащими за пределами диаграммы размаха) и заполним пропуски типа кузова, коробки передач и топлива наиболее часто встречающимися значениями для данной модели.

In [21]:
%%time

min_price = 300 # минимально возможная цена
k_price = 1.5 # коэффициент IQR для цены
len4 = len(data)

for model in data['ModelBrand'].unique():
    
    extract = data[data['ModelBrand'] == model] # выборка по модели, рассматриваемой на текущем шаге цикла
    q1 = extract['Price'].quantile(0.25) # квартиль 25%
    q3 = extract['Price'].quantile(0.75) # квартиль 75%
    iqr = q3 - q1 # межквартильный интервал
    low = q1 - k_price*iqr # минимальное нормальное значение
    if low < min_price:
        low = min_price
    high = q3 + k_price*iqr # максимальное нормальное значение
    extract = extract[(extract['Price']>=low) & (extract['Price']<=high)] # выборка по модели за исключением аномальных значений
    
    # заполнение пропусков в типе кузова
    vtype = extract['VehicleType'].mode()[0]
    extract['VehicleType']=extract['VehicleType'].fillna(vtype)
    
    # заполнение пропусков в типе коробки передач
    gearbox = extract['Gearbox'].mode()[0]
    extract['Gearbox']=extract['Gearbox'].fillna(gearbox)
    
    # заполнение пропусков в типе топлива
    ftype = extract['FuelType'].mode()[0]
    extract['FuelType']=extract['FuelType'].fillna(ftype)

    # замена в базе данных выборки по данной модели на выборку с удаленными аномальными значениями и заполненными пропусками:
    data = data[data['ModelBrand'] != model]
    data = pd.concat([data, extract])

print ('Удалено, %:', round((len4-len(data))/len4*100,1))

Удалено, %: 10.3
CPU times: user 1min 6s, sys: 484 ms, total: 1min 7s
Wall time: 1min 7s


In [22]:
# заполним пропуски в информации о ремонте на 'yes' - логично предположить, что при отсутствии ремонта, продавец обязательно
# указал бы это преимущество
data['NotRepaired']=data['NotRepaired'].fillna('yes')

Заменим аномальные значения мощности на медианные значения по модели и типу топлива.

In [23]:
# ограничим пределы мощности исходя из практически возможных значений
min_power = 50
max_power = 500

# дополнительно ограничим пределы мощности согласно диограмме размаха
k_power = 1.5 # коэффициент IQR для мощности
q1 = data['Power'].quantile(0.25)
q3 = data['Power'].quantile(0.75)       
iqr = q3 - q1
low = q1 - k_power*iqr # нижний предел мощности
if low < min_power:
    low = min_power    
high = q3 + k_power*iqr # верхний предел мощности
if high > max_power:
    high = max_power

In [24]:
# временно заменим аномальные значения мощности на пропуски
data.loc[data.Power < low, 'Power'] = np.nan
data.loc[data.Power > high, 'Power'] = np.nan

In [25]:
# создадим вспомогательный столбец 'FBM' - Fuel_Model_Brand
data['FMB'] = data['FuelType']+str('_')+data['ModelBrand']

In [26]:
%%time

# заменим пропуски (аномальные значения) мощности на медианные значения по модели и типу топлива
for fmb in data['FMB'].unique():
    power = data[(data['FMB']==fmb) & (data['Power'].isna()==False)]['Power'].median()
    data.loc[(data['FMB']==fmb) & (data['Power'].isna()==True), 'Power'] = data[(data['FMB']==fmb) & (data['Power'].isna()==True)]['Power'].fillna(power)

CPU times: user 56.4 s, sys: 51.9 ms, total: 56.4 s
Wall time: 57.6 s


In [27]:
# для редких моделей/типов топлива возможно значение power == NaN в ячейке выше:
data['Power'].isna().sum()

242

In [28]:
# заменим оставшиеся пропуски в столбце 'Power' на медианное значение мощности по всей базе данных
data['Power'] = data['Power'].fillna(data['Power'].median())

In [29]:
# еще раз проверим разброс значений мощности
print(data['Power'].min(),'...',data['Power'].max())

50.0 ... 247.0


-теперь ок.

In [30]:
# удалим вспомогательный столбец 'FMB'
data = data.drop(columns='FMB')

Предобработка данных произведена.  

In [31]:
data.head()

,Price,VehicleType,Gearbox,Power,Kilometer,FuelType,NotRepaired,RegYM,ModelBrand
0,480,sedan,manual,90.0,150000,petrol,yes,1993.000000,volkswagen_golf
2,1500,small,manual,75.0,150000,petrol,no,2001.500000,volkswagen_golf
8,999,small,manual,101.0,150000,petrol,yes,1998.000000,volkswagen_golf
57,1000,sedan,manual,101.0,150000,petrol,no,1998.833333,volkswagen_golf
72,6600,sedan,manual,105.0,150000,gasoline,no,2006.833333,volkswagen_golf


### 2. Обучение моделей

Т.к. будем использовать библиотеку LightGBM, то переведем тип категориальных признаков из 'object' в 'category'.  
Масштабируем численные признаки.  
В качестве метрики качества будем использовать RMSE (кв.корень из СКО).

In [32]:
# список категориальных признаков
categorical = ['VehicleType', 'Gearbox', 'FuelType', 'NotRepaired', 'ModelBrand']

In [33]:
data[categorical]=data[categorical].astype('category')

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 279586 entries, 0 to 254318
Data columns (total 9 columns):
Price          279586 non-null int64
VehicleType    279586 non-null category
Gearbox        279586 non-null category
Power          279586 non-null float64
Kilometer      279586 non-null int64
FuelType       279586 non-null category
NotRepaired    279586 non-null category
RegYM          279586 non-null float64
ModelBrand     279586 non-null category
dtypes: category(5), float64(2), int64(2)
memory usage: 12.3 MB


In [35]:
# инициализация scaler'a
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [36]:
# список признаков, которые будем масштабировать
numeric = ['Power', 'Kilometer', 'RegYM']

In [37]:
# настроим scaler по полной выборке
scaler.fit(data[numeric])

StandardScaler(copy=True, with_mean=True, with_std=True)

Выделим обучающую,валидационную и тестовую выборки. Затем применим scaler к каждой выборке отдельно.

In [38]:
# импорт функции train_test_split библиотеки sklearn
from sklearn.model_selection import train_test_split

In [39]:
# выделение тестовой выборки
data_1, test = train_test_split(data, test_size=0.2, random_state=12345)

In [40]:
# выделение обучающей и валидационной выборок
train, valid = train_test_split(data_1, test_size=0.25, random_state=12345)

In [41]:
print('Длина выборки')
print('- обучающая: ', len(train))
print('- валидационная: ', len(valid))
print('- тестовая: ', len(test))

Длина выборки
- обучающая:  167751
- валидационная:  55917
- тестовая:  55918


Применим scaler к каждой выборке:

In [42]:
train[numeric] = scaler.transform(train[numeric])
valid[numeric] = scaler.transform(valid[numeric])
test[numeric] = scaler.transform(test[numeric])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

Предупреждение SettingWithCopyWarning (выше) можно проигнорировать, т.к. датасеты train, valid и test изменились корректно:

In [43]:
train.head()

,Price,VehicleType,Gearbox,Power,Kilometer,FuelType,NotRepaired,RegYM,ModelBrand
128968,12500,small,manual,0.088687,-2.110794,petrol,no,1.523868,audi_a1
6756,3200,sedan,manual,-0.627414,0.548394,petrol,no,0.144911,volkswagen_golf
40955,1200,small,manual,-1.298759,0.548394,petrol,no,-0.409842,peugeot_2_reihe
281897,3790,bus,manual,-0.381254,0.548394,gasoline,yes,0.430212,opel_zafira
31570,950,small,manual,-1.522541,0.548394,petrol,yes,-0.806095,volkswagen_lupo


Данные готовы для обучения моделей.

In [44]:
# импорт библиотеки LightGBM
import lightgbm as lgb

In [45]:
# признаки
x_train = train.drop(columns='Price')
x_valid = valid.drop(columns='Price')
x_test = test.drop(columns='Price')

In [46]:
# целевой признак
y_train = train['Price']
y_valid = valid['Price']
y_test = test['Price']

In [47]:
# импорт функции расчета СКО (MSE) 
from sklearn.metrics import mean_squared_error as mse

### 2.1. Модель Gradient Boosting Decision Tree.

In [48]:
# заготовка таблицы вывода результата алгоритма 'gradient boosting decision tree'
columns = ['boosting_type','rmse','feature_fraction','learning_rate', 'iterations/n_estimators','max_depth','num_leaves','min_data_in_leaf','max_bin']
data_gbdt = []

In [49]:
pd.set_option('display.max_rows', None)

In [50]:
%%time

# Эти параметры менять не будем:
feature_fraction = 0.8
bagging_freq = 2
bagging_fraction = 0.5

# Подбор параметров (https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html).
# Т.к. даже при небольшом количестве вариантов значений параметров эта операция длится 33 мин, то реально рассмотренные
# варианты значений закомментируем. Оставим только вариант с наилучшими результатами.

for learning_rate in [0.1]: #[0.1,0.3,0.5]:
    for max_depth in [9]: #[7,8,9]:
        for num_leaves in [250]: #[150,200,250]:
            for min_data_in_leaf in [10]: #[5,10,15]:
                for max_bin in [100]: #[50,100]:
                    params = {} 
                    params['feature_fraction'] = feature_fraction
                    params['bagging_freq'] = bagging_freq
                    params['bagging_fraction'] = bagging_fraction
                    params['learning_rate'] = learning_rate
                    params['boosting_type'] = 'gbdt'
                    params['objective'] = 'regression'
                    params['metric'] = 'rmse'
                    params['max_depth'] = max_depth
                    params['num_leaves'] = num_leaves
                    params['min_data_in_leaf'] = min_data_in_leaf
                    params['max_bin'] = max_bin
                    params['n_estimators'] = 100
                    params['random_state'] = 12345
                    model = lgb.LGBMRegressor(**params)
                    model.fit(x_train, y_train)
                    y_pred = model.predict(x_valid)
                    rmse = round(mse(y_valid,y_pred)**0.5,1)
                    line_gbdt1 = ['gbdt',rmse, feature_fraction, learning_rate, 100, max_depth,num_leaves,min_data_in_leaf,max_bin]
                    data_gbdt.append(line_gbdt1)             

CPU times: user 55.3 s, sys: 317 ms, total: 55.6 s
Wall time: 56.1 s


In [51]:
# предварительная таблица результатов алгоритма 'gradient boosting decision tree'
summary_gbdt = pd.DataFrame(data=data_gbdt, columns=columns).sort_values(by='rmse').reset_index(drop=True)
summary_gbdt

,boosting_type,rmse,feature_fraction,learning_rate,iterations/n_estimators,max_depth,num_leaves,min_data_in_leaf,max_bin
0,gbdt,1466.5,0.8,0.1,100,9,250,10,100


In [52]:
# зафиксируем значения параметров лучшего результата на данном этапе
learning_rate1 = summary_gbdt.loc[0,'learning_rate']
max_depth1 = summary_gbdt.loc[0,'max_depth']
num_leaves1 =  summary_gbdt.loc[0,'num_leaves']
min_data_in_leaf1 = summary_gbdt.loc[0,'min_data_in_leaf']
max_bin1 = summary_gbdt.loc[0,'max_bin']
iterations1 = summary_gbdt.loc[0,'iterations/n_estimators']

In [53]:
%%time

# Попробуем улучшить результат, увеличив параметр 'num_leaves' до значения, рекомендованного документацией
# библиотеки (num_leaves несколько меньше, чем 2^(max_depth) - возьмем коэффициент 0.6) и увеличив число итераций в два раза (до 200).

iterations = 200
params['learning_rate'] = learning_rate1
params['max_depth'] = max_depth1
params['num_leaves'] = int(0.6*(2**max_depth1))
params['min_data_in_leaf'] = min_data_in_leaf1
params['max_bin'] = max_bin1
params['n_estimators'] = iterations
params['random_state'] = 12345
model = lgb.LGBMRegressor(**params)
model.fit(x_train, y_train)
y_pred = model.predict(x_valid)
rmse = round(mse(y_valid,y_pred)**0.5,1)
line_gbdt2 = ['gbdt',rmse, feature_fraction,learning_rate1, iterations, max_depth1,int(0.6*(2**max_depth1)),min_data_in_leaf1,max_bin1]
data_gbdt.append(line_gbdt2)

CPU times: user 11min 50s, sys: 2.77 s, total: 11min 53s
Wall time: 11min 57s


In [54]:
# окончательная таблица результатов алгоритма 'gradient boosting decision tree'
summary_gbdt2 = pd.DataFrame(data=data_gbdt, columns=columns).sort_values(by='rmse').reset_index(drop=True)
summary_gbdt2

,boosting_type,rmse,feature_fraction,learning_rate,iterations/n_estimators,max_depth,num_leaves,min_data_in_leaf,max_bin
0,gbdt,1453.8,0.8,0.1,200,9,307,10,100
1,gbdt,1466.5,0.8,0.1,100,9,250,10,100


Значение RMSE незначительно, но улучшилось.

In [55]:
# зафиксируем значения параметров лучшего результата 'gbdt'
learning_rate2 = summary_gbdt2.loc[0,'learning_rate']
max_depth2 = summary_gbdt2.loc[0,'max_depth']
num_leaves2 =  summary_gbdt2.loc[0,'num_leaves']
min_data_in_leaf2 = summary_gbdt2.loc[0,'min_data_in_leaf']
max_bin2 = summary_gbdt2.loc[0,'max_bin']
iterations2 = summary_gbdt2.loc[0,'iterations/n_estimators']

### 2.2. Модель Random Forest.

Применим алгоритм Random Forest с теми же параметрами, что и лучший вариант Decision Tree.  
Попробуем разные значения n_estimators.

In [56]:
# заготовка таблицы вывода результата алгоритма 'gradient boosting decision tree'
data_rf = []

In [57]:
%%time

for n_estimators in [iterations2]: #[iterations2, 50,100,150]: для экономии времени оставили параметр лучшего результата
    params = {}
    params['feature_fraction'] = feature_fraction
    params['bagging_freq'] = bagging_freq
    params['bagging_fraction'] = bagging_fraction
    params['learning_rate'] = learning_rate2
    params['boosting_type'] = 'rf'
    params['objective'] = 'regression'
    params['metric'] = 'rmse'
    params['max_depth'] = max_depth2
    params['num_leaves'] = num_leaves2
    params['min_data_in_leaf'] = min_data_in_leaf2
    params['max_bin'] = max_bin2
    params['n_estimators'] = n_estimators
    params['random_state'] = 12345
    model = lgb.LGBMRegressor(**params)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_valid)
    rmse = round(mse(y_valid,y_pred)**0.5,1)
    line_rf = ['random_forest',rmse, feature_fraction, learning_rate2, n_estimators, max_depth2, num_leaves2, min_data_in_leaf2, max_bin2]
    data_rf.append(line_rf)

CPU times: user 2min 11s, sys: 664 ms, total: 2min 12s
Wall time: 2min 12s


In [58]:
summary_rf = pd.DataFrame(data=data_rf, columns=columns).sort_values(by='rmse').reset_index(drop=True)
summary_rf

,boosting_type,rmse,feature_fraction,learning_rate,iterations/n_estimators,max_depth,num_leaves,min_data_in_leaf,max_bin
0,random_forest,1739.4,0.8,0.1,200,9,307,10,100


Как видим, модель 'random_forest' дала менее точный результат.

In [59]:
# зафиксируем значения параметра n_estimators лучшего результата 'random_forest'
n_estimators_rf = summary_rf.loc[0,'iterations/n_estimators']

### 2.3. Модель Dropouts meet Multiple Additive Regression Trees.

Применим алгоритм Dropouts meet Multiple Additive Regression Trees с теми же параметрами, что и лучший вариант Decision Tree. 

In [60]:
%%time

params = {}
params['feature_fraction'] = feature_fraction
params['bagging_freq'] = bagging_freq
params['bagging_fraction'] = bagging_fraction
params['learning_rate'] = learning_rate2
params['boosting_type'] = 'dart'
params['objective'] = 'regression'
params['metric'] = 'rmse'
params['max_depth'] = max_depth2
params['num_leaves'] = num_leaves2
params['min_data_in_leaf'] = min_data_in_leaf2
params['max_bin'] = max_bin2
params['n_estimators'] = iterations2
params['random_state'] = 12345
model = lgb.LGBMRegressor(**params)
model.fit(x_train, y_train)
y_pred = model.predict(x_valid)
rmse = round(mse(y_valid,y_pred)**0.5,1)
data_dart = [['dart',rmse, feature_fraction, learning_rate2, iterations2, max_depth2, num_leaves2, min_data_in_leaf2, max_bin2]] 

CPU times: user 6min 31s, sys: 1.43 s, total: 6min 32s
Wall time: 6min 34s


In [61]:
summary_dart = pd.DataFrame(data=data_dart, columns=columns)
summary_dart

,boosting_type,rmse,feature_fraction,learning_rate,iterations/n_estimators,max_depth,num_leaves,min_data_in_leaf,max_bin
0,dart,1535.4,0.8,0.1,200,9,307,10,100


### 3. Анализ моделей.

In [64]:
# таблица сравнения рассмотренных моделей на валидационной выборке
summary_valid = pd.concat([summary_gbdt2.head(1), summary_rf.head(1), summary_dart]).sort_values(by='rmse').reset_index(drop=True)
summary_valid

,boosting_type,rmse,feature_fraction,learning_rate,iterations/n_estimators,max_depth,num_leaves,min_data_in_leaf,max_bin
0,gbdt,1453.8,0.8,0.1,200,9,307,10,100
1,dart,1535.4,0.8,0.1,200,9,307,10,100
2,random_forest,1739.4,0.8,0.1,200,9,307,10,100


Проверим работу этих четырех моделей на тестовой выборке и сравним также скорость их работы - добавим столбец 'time'.

In [65]:
# заготовка таблицы вывода результата алгоритма 'gradient boosting decision tree'
columns_test = ['boosting_type','execution_time, sec','rmse','feature_fraction','learning_rate', 'iterations/n_estimators','max_depth','num_leaves','min_data_in_leaf','max_bin']
data_test = []

In [66]:
import time

In [67]:
# параметры, общие для всех моделей

params = {}
params['feature_fraction'] = feature_fraction
params['bagging_freq'] = bagging_freq
params['bagging_fraction'] = bagging_fraction
params['learning_rate'] = learning_rate2
params['objective'] = 'regression'
params['metric'] = 'rmse'
params['max_depth'] = max_depth2
params['num_leaves'] = num_leaves2
params['min_data_in_leaf'] = min_data_in_leaf2
params['max_bin'] = max_bin2
params['random_state'] = 12345
n_estimators = iterations2 # кроме модели 'random_forest', будет ниже учтено в цикле

In [68]:
# работа моделей 
for boosting_type in ['gbdt', 'rf', 'dart']:
    if boosting_type == 'rf':
        n_estimators = n_estimators_rf
    start = time.time()
    model = lgb.LGBMRegressor(**params, boosting_type=boosting_type, n_estimators=n_estimators)
    model.fit(x_train, y_train)
    y_pred_test = model.predict(x_test)
    end = time.time()
    execution_time = round((end-start),2)
    rmse = round(mse(y_test,y_pred_test)**0.5,1)
    line_test = [boosting_type, execution_time, rmse, feature_fraction, learning_rate2, n_estimators, max_depth2, num_leaves2, min_data_in_leaf2, max_bin2]
    data_test.append(line_test)

In [69]:
# таблица сравнения рассмотренных моделей на тестовой выборке
summary_test = pd.DataFrame(data = data_test, columns = columns_test).sort_values(by='rmse').reset_index(drop=True)
summary_test

,boosting_type,"execution_time, sec",rmse,feature_fraction,learning_rate,iterations/n_estimators,max_depth,num_leaves,min_data_in_leaf,max_bin
0,gbdt,96.59,1453.2,0.8,0.1,200,9,307,10,100
1,dart,268.00,1542.9,0.8,0.1,200,9,307,10,100
2,rf,147.01,1751.5,0.8,0.1,200,9,307,10,100


Значения метрики вполне соответствуют результатам работы моделей на валидационной выборке.

### Вывод.

Наилучшее значение метрики, как и наиболее короткое время работы показала модель Gradient Boosting Decision Tree.